In [1]:
import pandas as pd 
import numpy as np 
import neo4j


##import seaborn as sns
import warnings
from neo4j import GraphDatabase
warnings.filterwarnings('ignore')
import networkx as nx
from networkx.algorithms.link_analysis.pagerank_alg import pagerank
from collections import Counter
##from pandas_profiling import ProfileReport

In [2]:
url = "neo4j+s://9d0eb28d.databases.neo4j.io"
username = "neo4j"
password = "ZsZO3ya5YB8EcmVwmxYEqn6Amj_Z7R4t_6VDU5oMIeI"

driver = GraphDatabase.driver(url, auth=(username, password))
with driver.session() as session:
    result = session.run("MATCH (n) RETURN count(n)")
    count = result.single()[0]
    print(f"La conexión con Aura Neo4j se ha establecido correctamente. Hay {count} nodos en la base de datos.")

La conexión con Aura Neo4j se ha establecido correctamente. Hay 1169 nodos en la base de datos.


In [15]:
def _existencia_usuarios_nodo(tx, genero):
    results=tx.run("MATCH (g:Genre {nombre: $nombre}) RETURN g.nombre", nombre=genero)
    return(list(results))
with driver.session() as session:
        n=session.read_transaction(_existencia_usuarios_nodo,genero = "drama")
        print(n)

[<Record g.nombre='drama'>]


In [12]:
def usuarios_nodo(tx, id, p):
    tx.run("CREATE (:User {id: $id, p: $p})", id=id, p = p)
    
def existencia_usuarios_nodo(tx, id):
    result=tx.run("MATCH (u:User {id : $id}) return count(u)", id=id)
    return list(result)

def new_cliente(id, p):
    records = []
    with driver.session() as session:
        n=session.read_transaction( existencia_usuarios_nodo,id = id)
        print(n)
        if (n == 0 or n == None):
            session.write_transaction(usuarios_nodo,id = id, p = p)
        else: 
            print("No mms ya existe ese usuario")
    return None


    
new_cliente(800,34678)

[<Record count(u)=5>]
No mms ya existe ese usuario


In [19]:
#recomendaciones (pagerank)
from gettext import find


query = """MATCH MATCH (G:Genre {name: "Drama"})-[IN_GENRE]->(m:Movie)
RETURN G.name, m.name
"""
def películas_cliente(tx, password):
    result = tx.run(query, id=password)
    return list(result)                 
def pelis_drama(password):
    records=[]
    with driver.session() as session:
        result=session.read_transaction(películas_cliente, password)
        for record in result:
            records.append(record)
    print("Found {record} records!". format(record=len(records)))
    return records
a= pelis_drama(1)   #Resultados películas género drama

drama
Found  120062 records 


In [30]:
def create_networkx_graph(generos):
    
    val_count  = Counter([(res['m.nombre'],res['m2.nombre']) for res in pelis_drama(generos) ])
    if len(val_count)==0:
        print(f" no records for {password}")
    print(f"  no of  records for {password} {len(val_count)}")

    recs = pd.DataFrame([[a[0], a[1], val_count[a]] for a in val_count])

    G = nx.DiGraph()
    #recs= recs.head(20)
    total = len(val_count)
    for idx , rec in recs.iterrows():
        #print(f"{idx} of {total} loaded")
        G.add_edge(rec[0], rec[1], weight = rec[2])
    return G
G= create_networkx_graph(generos = ["drama","comedy"])

drama
Found  120062 records 
comedy
Found  145822 records 
  no of  records for ZsZO3ya5YB8EcmVwmxYEqn6Amj_Z7R4t_6VDU5oMIeI 144016


In [32]:
def run_page_rank(G, generos):
    pr=pagerank(G)
    sorted_pr = {k: v for k, v in sorted(pagerank(G).items(), key=lambda item: item[1], reverse = True)}
    df= pd.DataFrame(sorted_pr.items())
    df['rank'] = df[1].rank(method='dense', ascending=False)
    print(f"done calculating page rank")
    return df
df = run_page_rank(G,["drama","comedy"])

done calculating page rank


In [33]:
print(df.head(45))

                                                    0         1  rank
0                                      día de muertos  0.003408   1.0
1                                     at the drive-in  0.003408   1.0
2                               she's out of his mind  0.003408   1.0
3                                              tallos  0.003408   1.0
4                       l'étudiante et monsieur henri  0.003408   1.0
5                                              people  0.003408   1.0
6                                  a thousand junkies  0.003408   1.0
7                                     alles ist liebe  0.003408   1.0
8                                   hao nu shi ba jia  0.003408   1.0
9                                  miss india america  0.003408   1.0
10                                 kaamannana makkalu  0.003408   1.0
11                      ikite iru no wa hima tsubushi  0.003408   1.0
12                                          soap life  0.003408   1.0
13                  